In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys

import glob
import csv

import numpy as np
import pandas as pd

import tifffile as tiff

In [ ]:
main_path='/kaggle/input/hubmap-kidney-segmentation/'
train_path=main_path+'train/'
test_path=main_path+'test/'

In [ ]:
def get_shape(tiffile):# this function call fails on the first test image
    tif=tiff.TiffFile(tiffile)
    shape=tif.series[0].shape
    print('get shape', shape)

    #if shape[0]==1:#(1,1,3,x,y):
    #    shape=shape[3:]
    #elif shape[0]==3:#(3,x,y)
    #    shape=shape[1:]
    #elif shape[2]==3:#(x,y,3)
    #    shape=shape[0:2]
    return shape

def read_tiff(file_path):
    img = tiff.imread(file_path)
    print(file_path, img.shape)
    if img.shape[0]==1:#(1,1,3,x,y):
        img=np.moveaxis(img[0][0],0,-1)
    elif img.shape[0]==3:
        img=np.moveaxis(img,0,-1)

    print(file_path, img.shape)
    return img

def rle_encode_less_memory(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    This simplified method requires first and last pixel to be zero
    '''
    pixels = img.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)


In [ ]:
#You can use either of the two lines below. Both options work fine. It was a test to see of sample_submission.csv is a valid source of test_id
test_ids=[os.path.splitext(os.path.split(f)[1])[0] for f in glob.glob(test_path+'*.tiff')]
test_ids=list(pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv').id.values)
test_ids

In [ ]:
#generate random masks to have score (imitates the CNN inference)

try:
    np.random.seed(42)

    ids, preds = [], []
    for t_id in test_ids:
        test_tiff=test_path+t_id+'.tiff'
        shape=get_shape(test_tiff)#if I comment this like out the submission works fine. If the line is present the submission fails.
        shape=(10000,10000)
        print(test_tiff, shape)
        mask=np.zeros(shape, dtype=np.uint8)
        t=1/0
        for i in range(100):
            y=np.random.randint(0,shape[0]-1000)
            x=np.random.randint(0,shape[1]-1000)
            mask[y:y+1000, x:x+1000]=1

        encoded=rle_encode_less_memory(mask.T)

        ids.append(t_id)
        preds.append(encoded)   
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

else:
    print('We are cool')
    

In [ ]:
#If the previous cell fails this cell should never be exectuted and no submission.csv appear
#But this cell is executed even if the previous cell thows an exception

df = pd.DataFrame({'id':ids, 'predicted':preds})
df.to_csv('sub.csv', index=False)
df.to_csv('submission.csv', index=False)
df

In [ ]:
#this is the "hack" that I used to make the submission successful
#my assumption was that I must be forming the submission.csv incorrectly as kaggle was giving me "Error Scoring Submission"
#It turned out that Kaggle kept executing notebook cells after the exeption was thrown thus resulting in a malformed CSV with no private test submission
#This, in turn, resulted in perfectly valid puublic test score and 0 private test score.

if True:
    sample_sub=pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
    submission = pd.read_csv('sub.csv', index_col='id')
    sample_sub = pd.read_csv('/kaggle/input/hubmap-kidney-segmentation/test/sample_submission.csv', index_col='id')
    print(sample_sub.index.values)
    pub_ids = submission.index.values

    predictions = submission.values
    sample_sub.loc[pub_ids] = predictions
    sample_sub.to_csv('submission.csv')
    print('Done')
    # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
    # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session